In [10]:
import os
import numpy as np
from pyrecode.recode_reader import ReCoDeReader, merge_parts
from multiprocessing import Process, Manager
import time

%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [41]:
class ReaderNode:

    def __init__(self, pid, filename, refresh_rate=1):

        self._pid = pid
        self._filename = filename
        self._refresh_rate = refresh_rate
        self._seek_table = []

        self._reader = None
        self._is_initialized = False
        self._node_states = None
        self._node_shapes = None
        self._node_fetch_params = None
        self._node_fetch_results = None
        self._ny = None
        self._nx = None
        self._seek_position = 0

    def start(self, node_states, node_shapes, node_fetch_params, node_fetch_results):

        self._node_shapes = node_shapes
        self._node_states = node_states
        self._node_fetch_params = node_fetch_params
        self._node_fetch_results = node_fetch_results

        # wait for data and periodically check if its available
        self._node_states[self._pid] = 'init'
        while not self._is_initialized:
            time.sleep(self._refresh_rate)
            self._init()
            self._node_states[self._pid] = 'wait'
            
        while True:
            if self._node_states[self._pid] == 'open':
                self._open()
                self._node_states[self._pid] = 'wait'
                
            elif self._node_states[self._pid] == 'wait':
                time.sleep(self._refresh_rate)
                
            elif self._node_states[self._pid] == 'fetch':
                self._node_fetch_results[self._pid] = self._get_next()
                self._node_states[self._pid] = 'wait'
            elif self._node_states[self._pid] == 'close':
                self._close()
                self._node_states[self._pid] = 'wait'
            """
            elif self._node_states[self._pid] == 'fetch_file_position':
                self._node_fetch_params[self._pid] = self._reader.get_file_position()
                self._node_states[self._pid] = 'wait'
            """ 
                
    def _open(self):
        if self._is_initialized:
            self._reader = ReCoDeReader(self._filename, is_intermediate=True)
            self._reader.open(print_header=False)
            self._reader._fp.seek(self._seek_position, 0)

    def _init(self):
        # check if file exists, and read only when available
        if os.path.isfile(self._filename):
            self._reader = ReCoDeReader(self._filename, is_intermediate=True)
            if not self._is_initialized:
                self._reader.open(print_header=False)
                header = self._reader.get_header().as_dict()
                self._ny = header['ny']
                self._nx = header['nx']
                self._node_shapes[self._pid] = (self._ny, self._nx)
                self._reader.seek_to_frame_data()
                self._seek_position = self._reader.get_file_position()
                self._is_initialized = True
                # self._reader.close()
                
    def _get_shape(self):
        return (self._ny, self._nx)
                
    def _get_next(self):
        start_time_1 = datetime.now()
        s = np.zeros((self._ny, self._nx), dtype=np.uint16)
        count = 0
        if self._is_initialized:
            params = self._node_fetch_params[self._pid]
            for i in range(params['n_frames']):
                f = self._reader.get_next_frame()
                if f is not None:
                    frame_id = list(f.keys())[0]
                    if frame_id >= params['start_frame'] and frame_id < params['end_frame']:
                        frame_data = f[frame_id]['data'].todense()
                        s = np.add(s, frame_data)
                        count += 1
                        
        # print("4.3", datetime.now() - start_time_1)
        return {'count': count, 'sum': s}


    def _close(self):
        self._seek_position = self._reader.get_file_position()
        self._reader.close()


class ReCoDeViewer:

    def __init__(self, folder_path, base_filename, num_parts, frames_per_fetch, refresh_rate):

        self._folder_path = folder_path
        self._base_filename = base_filename

        self._num_parts = num_parts
        self._frames_per_fetch = frames_per_fetch
        self._refresh_rate = refresh_rate

        self._node_states = None
        self._node_shapes = None
        self._node_fetch_params = None
        self._node_fetch_results = None
        self._readers = None

        self._ny = None
        self._nx = None
        
        self._seek_positions = np.zeros(self._num_parts, dtype=np.uint64)
        self._is_closed = True
        self._iteration = 0

    def start(self):

        self._node_states = []
        
        manager = Manager()
        self._node_states = manager.dict()
        self._node_shapes = manager.dict()
        self._node_fetch_params = manager.dict()
        self._node_fetch_results = manager.dict()

        self._readers = []
        for index in range(self._num_parts):
            inter_file_name = os.path.join(self._folder_path, self._base_filename + '_part' + '{0:03d}'.format(index))
            reader = ReaderNode(index, inter_file_name, self._refresh_rate)
            self._readers.append(reader)
            
            self._node_states[index] = 'init'
            self._node_shapes[index] = None
            self._node_fetch_params[index] = None
            self._node_fetch_results[index] = None
            reader_process = Process(target=reader.start, args=(self._node_states, self._node_shapes,
                                                                self._node_fetch_params, self._node_fetch_results))
            self._readers.append(reader_process)
            reader_process.start()
            
            
    def get_shape(self):
        if self._node_states[0] == 'wait':
            return self._node_shapes[0]
        else:
            return None
    
    def _open_nodes(self):
        for index in range(self._num_parts):
            if self._node_states[index] == 'wait':
                self._node_states[index] = 'open'
            
    def _get_sum(self, frame_index):
        if self._node_states[frame_index] == 'wait':
            self._node_states[frame_index] = 'fetch'
            
    def _barrier(self):
        while True:
            states = [self._node_states[i] == 'wait' for i in range(self._num_parts)]
            if self._num_parts == np.sum(states):
                break
            time.sleep(0.01)
             
    def get_next_batch(self, n_frames):
        
        start_time = datetime.now()
        # open readers if closes
        if self._is_closed:
            for index in range(self._num_parts):
                self._open_nodes()
            self._is_closed = False
            # print("Opened nodes")
        self._barrier()
        # time.sleep(5)
        # print("1", datetime.now() - start_time)
        
        for index in range(self._num_parts):
            self._node_fetch_params[index] = {'n_frames': self._frames_per_fetch, 
                                              'start_frame': self._frames_per_fetch*self._iteration, 
                                              'end_frame': self._frames_per_fetch*(self._iteration+1)}
            self._get_sum(index)
        self._barrier()
        
        counts = 0
        sums = np.zeros(self._node_shapes[0], dtype=np.uint16)
        for index in range(self._num_parts):
            r = self._node_fetch_results[index]
            counts += r['count']
            sums = np.add(sums, r['sum'])          
        data = {'counts': counts, 'sum': sums}

        start_time = datetime.now()
        # close readers is no data is available
        if len(data) == 0:
            for index in range(self._num_parts):
                self._node_states[index] = 'close'
            self._is_closed = True
            # print("closed nodes")
        self._barrier()
        # print("5", datetime.now() - start_time)
        
        self._iteration += 1
        
        return data

    def close(self):
        for index in range(self._num_parts):
            if self._node_states[index] == 'wait':
                self._node_states[index] = 'close'

In [42]:
_data_folder = '/home/abhik/code/ReCoDe/scratch/live_view_test'
_dataset = ''
_tag = 'streampix_run_1'
_num_part_files = 10
_frames_per_fetch = 80
_frames_per_second = 40

viewer = ReCoDeViewer(os.path.join(_data_folder, _dataset), _tag + '.rc1', _num_part_files, _frames_per_fetch, 0.1)
viewer.start()

time_step = 0
time_point = 0

_shape = viewer.get_shape()
print("Reader ready, shape =", _shape)

frame_index = 0

while True:
    
    data = viewer.get_next_batch(_frames_per_fetch)
    
    n_frames = data['counts']
    view = data['sum']
    if n_frames > 0:
        is_stale = ''
        frame_index += len(frames)        
    else:
        is_stale = '\nWaiting for data...'
        
    checksum = np.sum(view)
    
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(view, vmax=1)
    t = time_step * _frames_per_second/_frames_per_fetch
    title_str = 'Elapsed Time = ' + str(t) +  ' seconds'\
                '\nShowing Sum of ' + str(n_frames) + ' frames' + \
                '\nchecksum = ' + str(checksum) + '\n' + is_stale
    ax.set(title=title_str)
    # fig.colorbar(im)
    plt.show()
    clear_output(wait=True)
    
    time_step += 1

Process Process-206:
Process Process-200:
Process Process-209:


KeyboardInterrupt: 

Process Process-208:
Process Process-204:
Process Process-203:
Process Process-202:
Process Process-201:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/abhik/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-41-7682c40de0a9>", line 40

In [3]:
_data_folder = '/scratch/loh/abhik/2Sep2020/live_view_test'
_num_part_files = 10

_frame_ids = {}
for index in range(_num_part_files):
    frame_ids = []
    intermediate_file_name = os.path.join(_data_folder, 
                                          'streampix_2k_60fps_run_4.rc1' + '_part' + '{0:03d}'.format(index))
    reader = ReCoDeReader(intermediate_file_name, is_intermediate=True)
    reader.open(print_header=False)
    header = reader.get_header().as_dict()
    for i in range(200):
        frame_data = reader.get_next_frame()
        frame_id = list(frame_data.keys())[0]
        frame_ids.append(frame_id)
    _frame_ids[index] = frame_ids
    reader.close()

In [4]:
print(_frame_ids)

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], 1: [3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30

In [17]:
_data_folder = '/home/abhik/code/ReCoDe/pyReCoDe/scratch'
_num_part_files = 3

_frame_ids = {}
for index in range(_num_part_files):
    frame_ids = []
    intermediate_file_name = os.path.join(_data_folder, 'run_1.rc1' + '_part' + '{0:03d}'.format(index))
    reader = ReCoDeReader(intermediate_file_name, is_intermediate=True)
    reader.open(print_header=True)
    header = reader.get_header().as_dict()
    for i in range(header['nz']):
        frame_data = reader.get_next_frame()
        frame_id = list(frame_data.keys())[0]
        frame_ids.append(frame_id)
    _frame_ids[index] = frame_ids
    reader.close()

ReCoDe Header
-------------
uid = 158966344846346
version_major = 0
version_minor = 2
is_intermediate = 1
reduction_level = 1
rc_operation_mode = 1
is_bit_packed = 1
target_bit_depth = 12
nx = 4096
ny = 512
nz = 217
frame_metadata_size = 0
num_non_standard_frame_metadata = 0
L2_statistics = 0
L4_centroiding = 0
compression_scheme = 0
compression_level = 1
source_file_type = 2
source_header_length = 1024
source_header_position = 0
source_file_name = /home/abhik/code/ReCoDe/scratch/temp2/Next_Stream.seq                                               
calibration_file_name = /scratch/loh/abhik/temp2/003.seq                                                                    
calibration_threshold_epsilon = 0
has_calibration_data = 1
frame_offset = 0
calibration_frame_offset = 0
num_calibration_frames = 1
source_bit_depth = 12
source_dtype = 0
target_dtype = 0
checksum = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
futures = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [18]:
for node_id in _frame_ids:
    print(node_id)
    print(_frame_ids[node_id])

0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 2436, 2437, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2467, 2468, 2469, 2470, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 3654, 3655, 3656, 3657, 3658, 3659, 3660, 3661, 3662, 3663, 3664, 3665, 3666, 3667, 3668, 3669, 3670, 3671, 3672, 3673, 3674, 3675, 3676, 3677, 3678, 3679, 3680, 3681, 3682, 3683, 3684, 3685, 3686, 3687, 3688, 3689, 3690, 3691, 3692, 3693, 3694, 3695, 3696, 3697, 3698, 4

In [19]:
merge_parts('/home/abhik/code/ReCoDe/pyReCoDe/scratch', 'run_1.rc1', 3)

In [20]:
recode_file_name = '../scratch/run_1.rc1'
reader = ReCoDeReader(recode_file_name, is_intermediate=False)
reader.open()

frame_data = reader.get_next_frame()
while frame_data is not None: 
    frame_id = list(frame_data.keys())[0]
    print(frame_id)
    frame_data = reader.get_next_frame()
reader.close()

ReCoDe Header
-------------
uid = 158966344846346
version_major = 0
version_minor = 2
is_intermediate = 1
reduction_level = 1
rc_operation_mode = 1
is_bit_packed = 1
target_bit_depth = 12
nx = 4096
ny = 512
nz = 217
frame_metadata_size = 0
num_non_standard_frame_metadata = 0
L2_statistics = 0
L4_centroiding = 0
compression_scheme = 0
compression_level = 1
source_file_type = 2
source_header_length = 1024
source_header_position = 0
source_file_name = /home/abhik/code/ReCoDe/scratch/temp2/Next_Stream.seq                                               
calibration_file_name = /scratch/loh/abhik/temp2/003.seq                                                                    
calibration_threshold_epsilon = 0
has_calibration_data = 1
frame_offset = 0
calibration_frame_offset = 0
num_calibration_frames = 1
source_bit_depth = 12
source_dtype = 0
target_dtype = 0
checksum = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
futures = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0